# Master the Art of Crafting, Optimizing, and Customizing Prompts for Mistral Models

*This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

Mistral AI foundation models are now generally available on Amazon Bedrock. In this demo notebook, we demonstrate the top use cases with Mistral models on Amazon Bedrock:

* Conversation Agents/Q&A
* Text Summarization and Classification
* Code Generation
* Function Calling

---
## Mistral Model Selection

### 1. Mistral 7B Instruct

- **Description:** A 7B dense Transformer model, fast-deployed and easily customizable. Small yet powerful for a variety of use cases.
- **Max Tokens:** 32K
- **Languages:** English
- **Supported Use Cases:** Text summarization, structuration, question answering, and code completion

### 2. Mixtral 8X7B Instruct

- **Description:** A 7B sparse Mixture-of-Experts model with stronger capabilities than Mistral 7B. Utilizes 12B active parameters out of 45B total.
- **Max Tokens:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Text summarization, structuration, question answering, and code completion

### 3. Mistral Small

- **Description:** Mistral Small is a highly efficient large language model optimized for high-volume, low-latency language-based tasks. It provides outstanding performance at a cost-effective price point. Key features of Mistral Small include RAG specialization, coding proficiency, and multilingual capabilities.
- **Max Tokens:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Optimized for straightforward tasks that can be performed in bulk, such as classification, customer support, or text generation


### 4. Mistral Large

- **Description:** Mistral AI’s most advanced large language model, Mistral Large is a cutting-edge text generation model with top-tier reasoning capabilities. Its precise instruction-following abilities enables application development and tech stack modernization at scale.
- **Max Tokens:** 32K
- **Languages:** Natively fluent in English, French, Spanish, German, and Italian
- **Supported Use Cases:** precise instruction following, text summarization, translation, complex multilingual reasoning tasks, math and coding tasks including code generation


For more information, refer to the following links:

1. [Mistral Model Selection Guide](https://docs.mistral.ai/guides/model-selection/)
2. [Amazon Bedrock Pricing Page](https://aws.amazon.com/bedrock/pricing/)

---

In [2]:
mistral7b_id = "mistral.mistral-7b-instruct-v0:2"
mixtral8x7b_id = "mistral.mixtral-8x7b-instruct-v0:1"
mistral_large_id = "mistral.mistral-large-2402-v1:0"
mistral_small_id = "mistral.mistral-small-2402-v1:0"

---

## Bedrock APIs

### [Bedrock's InvokeModel API](https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModel_MistralAi_section.html)

#### InvokeModel API supported papameters

The Mistral AI models have the following inference parameters.


```
{
    "prompt": string,
    "max_tokens" : int,
    "stop" : [string],    
    "temperature": float,
    "top_p": float,
    "top_k": int
}
```

The Mistral AI models have the following inference parameters:

- Temperature - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- Top P - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
- Top K - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- Maximum Length - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- Stop sequences - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

### [Bedrock's Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_Converse_Mistral_section.html)

#### Benefits of using Converse API


- Natively adds function calling (tool use)
- Simplifies prompt syntax for certain models - e.g: [INST] tag for Mistral
- Unifies system prompts across models
- Common parameters are separated from model-specific parameters
- Does not cover image generation and embeddings
- Unifies API for multi-turn conversations

---


In [3]:
!pip install --upgrade --quiet boto3 botocore

In [4]:
import boto3
import json

class LLM:
    def __init__(self, model_id):
        self.model_id = model_id
        self.bedrock = boto3.client(service_name="bedrock-runtime")
        
    def invoke(self, prompt, temperature=0.0, max_tokens=128):
        body = json.dumps({
            "temperature": temperature,
            "max_tokens": max_tokens,
            "prompt": prompt, 
            "stop": ["</s>"]
        })
        response = self.bedrock.invoke_model(
            body=body, 
            modelId=self.model_id)

        response_body = json.loads(response.get("body").read())
        return response_body['outputs'][0]['text']
    
    
    
llm = LLM(mixtral8x7b_id)

In [5]:
prompt = "<s>[INST] What is the capital of France? [/INST]"

In [6]:
response_text = llm.invoke(
    prompt,
    temperature=0.0,
    max_tokens=128,
)
print(response_text)

 The capital of France is Paris. It's located in the north-central part of the country and is one of the most populous and visited cities in the European Union. Paris is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and the Champs-Élysées. It's also famous for its cuisine, fashion, and art scene. The city has a rich history and has played a significant role in shaping Western civilization.


---

## Conversation Agents/Q&A

Mistral models can be utilized for developing powerful chatbots due to their ability to understand and generate human-like responses, while optimized for low latency, high throughput and cost efficiency.

---

### Example: Chatbot with persona

Chatbot with persona AI assistant will play the role of AWS customer service assistant.

When implementing a chatbot, it needs to retain the context of previous interactions. Let's begin incorporating the chat history into the prompt and storing the history in memory.

In [7]:
from typing import Dict, List
from IPython.display import Markdown, display

def chat_history_to_string(memory):
    history_str = ""
    for chat_item in memory:
        role = chat_item.get("role", "")
        content = chat_item.get("content", "")
        history_str += f"{role}: {content}\n\n"
    return history_str.strip()

def format_conversation(user_input: str, memory: List[Dict[str, str]] = []) -> str:
    
    history = chat_history_to_string(memory)
    
    prompt = f"""
    <s>[INST] You are a knowledgeable helpful AWS customer service assistant. You are helpful and provide general guidance from the context less than 100 words in the scope.[/INST]
    {history} 
    <s>[INST] {user_input} [/INST]
    """
    return prompt

def chat_with_agent(user_input: str, memory: List[Dict[str, str]]):
    response = llm.invoke(
        format_conversation(user_input, memory),
        temperature=0.0,
        max_tokens=512,
    )
    display(Markdown(response))
    memory.append({"role": "customer", "content": user_input})
    memory.append({"role": "assistant", "content": response})

In [8]:
memory = []
chat_with_agent("Hi there", memory)

 Hello! Welcome to AWS customer service. I'm here to help. What can I assist you with today regarding AWS? Please keep your questions concise for a prompt response.

In [9]:
chat_with_agent("How to select an EC2 instance type?", memory)

 To select an EC2 instance type, consider:

1. Workload: Compute-intensive, memory-intensive, or storage-intensive.
2. Instance family: General purpose, compute optimized, memory optimized, or storage optimized.
3. Size: Number of vCPUs and memory.
4. Pricing: On-demand, reserved, or spot instances.
5. Additional features: GPU, FPGA, or bare metal.

For more details, visit: [EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/)

In [10]:
chat_with_agent("Cool. Will that work for my Linux workload?", memory)


    Absolutely! The EC2 instance selection guidance I provided is generally applicable to Linux workloads. AWS offers a wide variety of instances optimized for different Linux distributions. You can choose an instance type based on your specific requirements for compute, memory, and storage, as well as your budget.

For more information on Linux instances, visit: [Linux on AWS](https://aws.amazon.com/linux/)

Please let me know if you have any other questions or concerns. I'm here to help!

In [11]:
chat_with_agent("That's all. Thank you.", memory)


    You're welcome! I'm glad I could help. If you have any more questions or concerns in the future, don't hesitate to reach out. Have a great day! 😊

**Let's ask a question that is not specialty of this persona.**

In [12]:
chat_with_agent("How to fix my car?", [])

 I'm sorry for any confusion, but I'm an AWS customer service assistant and I don't have the ability to diagnose or fix car problems. I can certainly help you with any questions or issues you have related to Amazon Web Services. Is there something specific you'd like assistance with in regards to AWS?

---

## Text Summarization and Classification
Text summarization is a crucial task that extracts the most important information from text based documents while retaining its core meaning. With a context window up to 32k tokens, organizations can use Mistral models to streamline their various document summarization needs from summarizing news articles and research papers to distilling critical information from long documents that enable downstream solutions such as document classification, question and answering, and decision support.

---

### Example 1: Summarization

This example provides a summary of the blog post content [Mistral AI models now available on Amazon Bedrock](https://aws.amazon.com/blogs/aws/mistral-ai-models-now-available-on-amazon-bedrock/) and proposes relevant questions and answers in a markdown format.

In [13]:
import requests
response = requests.get('https://aws.amazon.com/blogs/aws/mistral-ai-models-now-available-on-amazon-bedrock/')
blog = response.text

In [14]:
message = f"""
<s>[INST] Your task is to write a summary about a blog post. 
When presented with the blog post, come up with quiz/answers to ask the viewers. 
Write the summary in the markdown format. 

# Blog post: 
{blog}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes presented in the blog post.

## Quiz: 
Generate three distinct questions that can be asked about the blog post. For each question:
- After "Question: ", describe the problem 
- After "Choices: ", possible answers (single choice)
    - A 
    - B
    - C
- After "Answer: ", show the correct choice

[/INST]
"""

In [15]:
response = llm.invoke(
        message,
        temperature=0.0,
        max_tokens=512,
    )
print(response)


# Summary:
In this blog post, AWS announced the availability of Mistral AI models on Amazon Bedrock. Mistral AI offers a balance of cost and performance, fast inference speed, transparency and trust, and is accessible to a wide range of users. The two high-performing Mistral AI models, Mistral 7B and Mixtral 8x7B, are now available on Amazon Bedrock, joining other leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon. This integration provides users the flexibility to choose optimal high-performing foundation models in Amazon Bedrock.

# Quiz:

Question: What are the two high-performing Mistral AI models available on Amazon Bedrock?

Choices:
- A. Mistral 7B and Mixtral 8x7B
- B. Mistral 6B and Mixtral 7x7B
- C. Mistral 8B and Mixtral 9x7B

Answer: A

Question: Which AI companies are currently offering foundation models on Amazon Bedrock?

Choices:
- A. AI21 Labs, Anthropic, Cohere, Meta, Stability AI, Amazon, and Mistral AI
- B. AI21 Labs, Anthropic, 

### Example 2: Classification

Data classification is crucial for ensuring the proper protection, management, and controlled access of an organization's information assets based on their sensitivity levels. In the following example, we provided the LLM with different data sensitivity levels and their definitions, allowing it to classify user inquiries accordingly.

In [16]:
def fill_classification_template(user_inquery):
    query = f"""<s>[INST]
You are an AI assistant tasked with classifying data based on its sensitivity level. The sensitivity levels and their definitions are:

Sensitive: Data that is to have the most limited access and requires a high degree of integrity. This is typically data that will do the most damage to the organization should it be disclosed.
Confidential: Data that might be less restrictive within the company but might cause damage if disclosed.
Private: Private data is usually compartmental data that might not do the company damage but must be kept private for other reasons. Human resources data is one example of data that can be classified as private.
Proprietary: Proprietary data is data that is disclosed outside the company on a limited basis or contains information that could reduce the company's competitive advantage, such as the technical specifications of a new product.
Public: Public data is the least sensitive data used by the company and would cause the least harm if disclosed. This could be anything from data used for marketing to the number of employees in the company.

For each user inquery provided, classify it into one of the above sensitivity levels. Do not include the word "Category". Do not provide explanations or notes.

<<<
Inquiry: {user_inquery}
>>>

[/INST]"""
    return query

In [17]:
user_inqueries = [
    "I'm an HR recruiter. What data classifiction category are resumes gathered based on referral by employees?",
    "I require the financial statements for the past three fiscal years.",
    "I need access to the personnel files containing employee social security numbers and financial information."
]

for user_inquery in user_inqueries:
    response = llm.invoke(
        fill_classification_template(user_inquery),
        temperature=0.0,
        max_tokens=128,
    )
    print(f"User inquery: {user_inquery}\n")
    print(f"Category: {response}\n")

User inquery: I'm an HR recruiter. What data classifiction category are resumes gathered based on referral by employees?

Category:  Private

User inquery: I require the financial statements for the past three fiscal years.

Category:  Confidential

User inquery: I need access to the personnel files containing employee social security numbers and financial information.

Category:  Sensitive



--- 

## Code Generation

Mistral models can help developers write code faster and more efficiently, and enhance code quality by automating some of the repetitive coding tasks, such as generating boilerplate code, inserting comments, suggesting code snippets, or translating code from one programming language to another. By fine-tuning Mistral models on legacy languages, Mistral models can empower organizations to complete their language upgrade and other legacy infrastructure transformation initiatives quickly and cost effectively.

---

### Example 1: Python Code Generation

In the earlier classification example, we demonstrated the model's capability to classify data sensitivity. For the first code generation example, we can instruct the Mistral model to write a rule-based data classification Python function.

In [18]:
message = """
<s>[INST]
You are an experienced Python developer tasked with creating data classification function.
The function should analyze the input text and determine the appropriate classification category based on predefined rules or patterns.

1. Prompt the user to enter text for data classification. 
2. Implement the following classification rules. 
    - Sensitive: if the input text contains "password", "social security number", "credit card number"
    - Confidential: if the input text contains "confidential", "internal use only"
    - Private: if the input text contains "private", "personal"
    - Proprietary: if the input text contains "proprietary", "trade secret"
    - Public: Does not contain any sensitive, confidential, private, or proprietary information.
2. The function should return the classified category as a string (e.g., "Sensitive", "Confidential", "Private", "Proprietary", or "Public").
3. User Interface:
    - Create a simple command-line interface (CLI) that prompts the user to enter a string of text.
    - Provide clear instructions on how to use the program.
4. Error Handling: Implement error handling mechanisms to gracefully handle invalid inputs or exceptions.

[/INST]
"""

In [19]:
response = llm.invoke(
        message,
        temperature=0.0,
        max_tokens=1024,
    )
display(Markdown(response))

Here's a Python function that meets the requirements:

```python
import re

def classify_data(input_text):
    classifications = {
        "Sensitive": re.search(r"password|social security number|credit card number", input_text),
        "Confidential": re.search(r"confidential|internal use only", input_text),
        "Private": re.search(r"private|personal", input_text),
        "Proprietary": re.search(r"proprietary|trade secret", input_text),
        "Public": not any(classification for classification in (
            classifications.values()
            if classification is not None
        ))
    }

    for category, search_result in classifications.items():
        if search_result is not None:
            return category

def main():
    print("Data Classification Program")
    print("--------------------------")
    input_text = input("Enter the text to classify: ")
    classification = classify_data(input_text)
    print(f"\nClassification: {classification.capitalize() if classification else 'Public'}")

if __name__ == "__main__":
    main()
```

This script defines a `classify_data` function that takes a string as input and returns a classification category based on the predefined rules. The `main` function handles user input and displays the classification result.

To use this script, simply run it and enter the text you want to classify when prompted. The script will then output the classification category. If the input text does not contain any sensitive, confidential, private, or proprietary information, the script will classify it as "Public".

In [20]:
# copy the code from the response and run it!

### Example 2: Data science

A real world scenario is given a csv file and let the LLM to generate code to analyze the data, generate some visualizations and clean up the data. 

In [21]:
import csv

data = [
    ["book_id", "title", "author", "genre", "publish_date", "publisher", "pages", "rating"],
    [1, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction', '1925-04-10', "Charles Scribner's Sons", 180, 4.3],
    [2, 'To Kill a Mockingbird', 'Harper Lee', 'Fiction', '1960-07-11', 'J. B. Lippincott & Co.', 281, 4.2],
    [3, '1984', 'George Orwell', 'Fiction', '1949-06-08', 'Secker & Warburg', 328, 4.1],
    [4, 'Pride and Prejudice', 'Jane Austen', 'Fiction', '1813-01-28', 'Thomas Egerton', 279, 4.4],
    [5, 'The Catcher in the Rye', 'J. D. Salinger', 'Fiction', '1951-07-16', 'Little Brown and Company', 214, 3.8],
    [6, 'The Hobbit', 'J.R.R. Tolkien', 'Fantasy', '1937-09-21', 'George Allen & Unwin', 310, 4.5],
    [7, 'The Lord of the Rings', 'J.R.R. Tolkien', 'Fantasy', '1954-07-29', 'George Allen & Unwin', 1178, 4.7],
    [8, 'Harry Potter and the Sorcerer\'s Stone', 'J.K. Rowling', 'Fantasy', '1997-06-26', 'Bloomsbury', 309, 4.5],
    [9, 'The Da Vinci Code', 'Dan Brown', 'Mystery', '2003-03-18', 'Doubleday', 454, 3.6],
    [10, 'Angels & Demons', 'Dan Brown', 'Mystery', '2000-05-01', 'Atria Books', 620, 3.9],
    [11, 'The Girl on the Train', 'Paula Hawkins', 'Mystery', '2015-01-13', 'Riverhead Books', 323, 3.8],
    [12, 'Gone Girl', 'Gillian Flynn', 'Mystery', '2012-06-05', 'Crown Publishing Group', 422, 4.1],
    [13, 'The Notebook', 'Nicholas Sparks', 'Romance', '1996-10-01', 'Warner Books', 224, 4.0],
    [14, 'Outlander', 'Diana Gabaldon', 'Romance', '1991-06-01', 'Delacorte Press', 850, 4.2],
    [15, 'The Fault in Our Stars', 'John Green', 'Romance', '2012-01-10', 'Dutton Books', 313, 4.4]
]

# Write data to books.csv
with open('books.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("books.csv has been created!")

books.csv has been created!


In [22]:
message = """<s>[INST]You are a data analyst working with a dataset containing information about various books. The dataset is in the form of a CSV file named 'books.csv' and includes the following columns:

- 'book_id': Unique identifier for each book
- 'title': Title of the book
- 'author': Author of the book
- 'genre': Genre of the book (e.g., Fiction, Non-Fiction, Mystery, Romance)
- 'publish_date': Publication date of the book
- 'publisher': Publisher of the book
- 'pages': Number of pages in the book
- 'rating': Average rating of the book on a scale of 1 to 5

Your task is to write Python code using pandas library to perform the following data analysis and exploration:

1. Load the 'books.csv' file into a pandas DataFrame.
2. Display the first few rows of the DataFrame to get an overview of the data.
3. Check for any missing values in the DataFrame and handle them appropriately (e.g., drop rows or fill with a suitable value).
4. Convert the 'publish_date' column to a datetime format.
5. Create a new column 'age' that calculates the number of years since the book was published.
6. Group the books by genre and calculate the mean rating for each genre.
7. Identify the top 5 books with the highest ratings.
8. Create a scatter plot showing the relationship between the number of pages and the book rating.
9. Generate a bar chart displaying the count of books published by each publisher.
10. Export the processed and cleaned DataFrame to a new CSV file named 'books_cleaned.csv'.

Your code should be well-commented, easy to read, and follow best practices for data analysis and visualization using pandas and matplotlib (or any other suitable library). Feel free to add any additional analysis or visualizations that you think would be useful for exploring and understanding the book dataset.

[/INST]
"""

In [23]:
response = llm.invoke(
        message,
        temperature=0.0,
        max_tokens=1024,
    )
print(response)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# 1. Load the 'books.csv' file into a pandas DataFrame.
df = pd.read_csv('books.csv')

# 2. Display the first few rows of the DataFrame to get an overview of the data.
print(df.head())

# 3. Check for any missing values in the DataFrame and handle them appropriately (e.g., drop rows or fill with a suitable value).
missing_values = df.isnull().sum()
print("\nMissing Values:")
print(missing_values)

# Since there are no missing values, no need to drop or fill any values.

# 4. Convert the 'publish_date' column to a datetime format.
df['publish_date'] = pd.to_datetime(df['publish_date'])

# 5. Create a new column 'age' that calculates the number of years since the book was published.
today = datetime.today()
df['age'] = (today.year - df['publish_date'].dt.year) - \
            ((today.month, today.day) < (df['publish_date'].dt.month, df['publish_date'].dt.day))

# 6. Group the books by ge

In [24]:
# copy the code from the response and run it!

---

###  Function Calling
Function calling is the ability to reliably connect a large language model (LLM) to external tools and enable effective tool usage and interaction with external APIs. Mistral models provide the ability for building LLM powered chatbots or agents that need to retrieve context for the model or interact with external tools by converting natural language into API calls to retrieve specific domain knowledge. From conversational agents and math problem solving to API integration and information extraction, multiple use cases can benefit from this capability provided by Mistral models

---

### Example: Custom tool

Let's start by building a custom tool to give models access to the current date, a piece of information that is very relevant to answer queries about the real world, and that LLMs do not have access to, as they lack awareness of present vs. past time.

In [25]:
import io
import os
import sys
import json
import boto3
from datetime import date, datetime

Two functions have been defined:

- `get_age_from_birthdate` calculates the age based on the provided birthdate.

- `get_ticket_price_by_age` takes a person's age as input and calculates the appropriate ticket price based on that age. It then returns the calculated ticket price.

In [26]:
def get_age_from_birthdate(birthdate: str) -> int:
    birthdate = datetime.strptime(birthdate, "%Y-%m-%d").date()
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

def get_ticket_price_by_age(age: int) -> int:
    if age <= 3:
        return 0
    elif age >= 60:
        return 30
    return 60


We can write the Open API schema to describe the two functions.

In [27]:
tool_config = {
    "toolChoice": {"auto": {}},
    "tools": [
        {
            "toolSpec": {
                "name": "get_age_from_birthdate",
                "description": "Calculates the age based on the provided birthdate.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "birthdate": {
                                "type": "string",
                                "description": "The birthdate in the format YYYY-MM-DD."
                            }
                        },
                        "required": ["birthdate"]
                    }
                }
            }
        },
        {
            "toolSpec": {
                "name": "get_ticket_price_by_age",
                "description": "Returns the ticket price based on the provided age.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "age": {
                                "type": "integer",
                                "description": "The age of the person."
                            }
                        },
                        "required": ["age"]
                    }
                }
            }
        }
    ]
}

In [28]:
def use_tool(messages):
    tool_use = messages[-1]["content"][-1].get("toolUse")
    if tool_use:
        tool_name = tool_use["name"]
        tool_input = tool_use["input"]
        print(f"Tool Name: {tool_name}")
        print(f"Tool Input: {json.dumps(tool_input, indent=2)}")

        # Process the tool call
        tool_result = ""
        if tool_name == "get_age_from_birthdate":
             tool_result = get_age_from_birthdate(tool_input["birthdate"])
        elif tool_name == "get_ticket_price_by_age":
             tool_result = get_ticket_price_by_age(tool_input["age"])
        print(f"Tool Result: {tool_result} \n")
        message = {
            "role": "user",
            "content": [
                {
                    "toolResult": {
                        "toolUseId": tool_use["toolUseId"],
                        "content": [
                            {"text": json.dumps(tool_result)}
                        ],
                        "status": "success",
                    }
                }
            ],
        }

        return message

Below is the prompt template implementing the function calling, where we pass the tools available to the LLM, instruct the LLM to generate both reasoning traces and task-specific actions in an interleaved manner.

In [29]:
SYSTEM_PROMPT = f"""
You are a ticket price agent for an event venue. Your job is to provide the ticket price based on the customer's age.
You have access to a set of tools, but only use them when needed.  
If you do not have enough information to use a tool correctly, ask a user follow up questions to get the required inputs.
Do not call any of the tools unless you have the required data from a user.
"""

Below is the step-by-step execution of the function calling.

In [30]:
bedrock_client = boto3.client(service_name="bedrock-runtime")

def process_message(messages):
    """Process the given messages and generate a response using the Bedrock model."""
    system_prompts = [{"text": SYSTEM_PROMPT}]

    inference_config = {"temperature": 0.0, "maxTokens": 400}
    response = bedrock_client.converse(
        modelId=mistral_large_id,
        messages=messages,
        system=system_prompts,
        toolConfig=tool_config,
        inferenceConfig=inference_config,
    )
    message = response["output"]["message"]
    stop_reason = response.get("stopReason")
    return message, stop_reason

In [31]:
def ask_bot(user_input):
    messages = []
    messages.append(
        {
            "role": "user", "content": [{"text": user_input}]
        }
    )
    bot_response, stop_reason = process_message(messages)
    
    while stop_reason == "tool_use":
        messages.append(bot_response)
        tool_response = use_tool(messages)
        messages.append(tool_response)
        bot_response, stop_reason = process_message(messages)
    return bot_response["content"][0]["text"]

In [32]:
question = "I was born on Jan 02, 1999. How much do I need to pay for the ticket?"

display(Markdown("---"))
display(Markdown('**Function calling step-by-step:**'))
response = ask_bot(question)

display(Markdown("---"))
display(Markdown("**Final anawer:**"))
print(response)

---

**Function calling step-by-step:**

Tool Name: get_age_from_birthdate
Tool Input: {
  "birthdate": "1999-01-02"
}
Tool Result: 25 

Tool Name: get_ticket_price_by_age
Tool Input: {
  "age": 25
}
Tool Result: 60 



---

**Final anawer:**

Based on your age, the ticket price for the event is $60.


---

## Distributors

- Amazon Web Services
- Mistral AI

---